In [ ]:
import time_series_utils as utils
from pathlib import Path

import numpy as np

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
time_steps = 10

# Import data instead of generation
X, targets, y = utils.generate_data(time_steps, 2, using_classes=True)

In [ ]:
train_X, train_targets, train_y, val_X, val_targets, val_y = utils.split_df_to_train_test(X, targets, y, split_rate=0.6)
val_X, val_targets, val_y, test_X, test_targets, test_y = utils.split_df_to_train_test(val_X, val_targets, val_y, split_rate=0.5)

In [ ]:
amount_of_features = train_X[0].shape[1]

In [ ]:
def scale(X, scaler):
    return scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X.reshape(-1, train_X.shape[-1]))

In [ ]:
n_hidden_layers_list = [1, 2, 3]
n_neurons_list = [64, 128, 256]
n_neurons_last_layer_list = [64, 128, 256]

best_config = {
    'n_hidden_layers': -1,
    'n_neurons': -1,
    'n_neurons_last_layer': -1,
    'loss': 100000,
    'model': None,
    'model_history': None
}

sklearn_weights = class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)
weights = dict(enumerate(sklearn_weights))

Path("lstm-clf-models").mkdir(parents=True, exist_ok=True)

for n_hidden_layers in n_hidden_layers_list:
    for n_neurons in n_neurons_list:
        for n_neurons_last_layer in n_neurons_last_layer_list:
            print('Training {} hidden layers with {} neurons ({} neurons in last layer)'.format(n_hidden_layers, n_neurons, n_neurons_last_layer))
            es = EarlyStopping(monitor='val_loss', patience=50)
            mcp_save = ModelCheckpoint('lstm-clf-models/model-cp.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

            model = utils.CondLSTMModel(time_steps, train_X.shape[-1], n_neurons=n_neurons, n_hidden_layers=n_hidden_layers, n_neurons_last_layer=n_neurons_last_layer, using_classes=True)
            model.call([scale(train_X, scaler), train_targets])
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            history = model.fit(x=[scale(train_X, scaler), train_targets], y=train_y, validation_data=([scale(val_X, scaler), val_targets]), epochs=1000, verbose=0, callbacks=[es, mcp_save], class_weight=weights)
            
            model.load_weights('lstm-clf-models/model-cp.h5')
            
            metrics = model.evaluate([scale(val_X, scaler), val_targets], val_y, verbose=0)
            loss = metrics[0]
            print('Training done. Val loss: {:.2f}'.format(loss))
            
            if best_config['loss'] >= loss:
                print('Best setup so far! (val loss: {})'.format(loss))
                best_config['n_hidden_layers'] = n_hidden_layers
                best_config['n_neurons'] = n_neurons
                best_config['n_neurons_last_layer'] = n_neurons_last_layer
                best_config['loss'] = loss
                best_config['model'] = model
                best_config['model_history'] = history
                model.save_weights('lstm-clf-models/model.h5')

In [ ]:
history = best_config['model_history']
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
model = utils.CondLSTMModel(time_steps, train_X.shape[-1], n_neurons=best_config['n_neurons'], n_hidden_layers=best_config['n_hidden_layers'], n_neurons_last_layer=best_config['n_neurons_last_layer'])
model.call([scale(train_X, scaler), train_targets])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.load_weights('lstm-clf-models/model.h5')
model.evaluate([scale(test_X, scaler), test_targets], test_y)